In [1]:
import pandas as pd # standard python data library
import geopandas as gp # the geo-version of pandas
import numpy as np 
import os
import fiona
from statistics import mean, median
from pandas import read_csv
gp.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw' #To load KML files
import string
import xml.etree.ElementTree as et

Utah
----
Election results from OpenElections (http://openelections.net/)
Shapefile from the Utah Automated Geographic Reference Center (https://gis.utah.gov/data/political/voter-precincts/)

Several transcription errors were corrected from OpenElections using their raw results gathered from counties (https://github.com/openelections/openelections-sources-ut/tree/master/2018%20General%20Election). OE results in Rich and Morgan Counties were based on non-final reports, which were collected from county sources.

Emery County - 8 "canvas votes" were distributed to precincts.

Weber County - 6 precincts had their results suppressed, since all cast fewer than 15 votes, for a total of 42 votes. These were distributed based on the number of votes cast and the reported vote in 2016, which was not suppressed.

G18USSRROM - Mitt Romney (Republican Party)
G18USSDWIL - Jenny Wilson (Democratic Party)
G18USSIMCC - Reed C. McCandless (Independent American Party)
G18USSLBOW - Craig R. Bowden (Libertarian Party)
G18USSCAAL - Tim Aalders (Conservative Party)
G18USSOWRI - Write-in Votes

In [2]:
vest_ut_18 = gp.read_file("./raw-from-source/VEST/ut_2018/ut_2018.shp")

In [3]:
print(vest_ut_18.head())

   CountyID mergeid uniqueid  G18USSRROM  G18USSDWIL  G18USSIMCC  G18USSLBOW  \
0         1    BV01   1-BV01         402          46           8           9   
1         1    BV02   1-BV02         310          46           3          10   
2         1    BV03   1-BV03         244          30           0           2   
3         1    BV04   1-BV04          84           7           1           4   
4         1    GR01   1-GR01          56           9           6           1   

   G18USSCAAL  G18USSOWRI                                           geometry  
0          19           1  POLYGON ((358362.560 4238423.840, 358372.350 4...  
1          17           0  POLYGON ((355866.750 4240506.080, 356860.230 4...  
2          12           2  POLYGON ((358902.400 4261055.000, 358909.700 4...  
3          23           0  POLYGON ((367866.500 4262536.100, 367842.500 4...  
4           1           1  POLYGON ((348977.460 4223848.220, 348973.090 4...  


In [4]:
ut_election = pd.read_csv("./raw-from-source/Open_Elections/openelections-data-ut-master/2018/20181106__ut__general__precinct.csv")

In [5]:
fips_file = pd.read_csv("./raw-from-source/FIPS/US_FIPS_Codes.csv")
fips_file = fips_file[fips_file["State"]=="Utah"]
fips_file["FIPS County"]=fips_file["FIPS County"].astype(str)
fips_file["FIPS County"]=fips_file["FIPS County"].str.zfill(3)
ut_fips_dict = dict(zip(fips_file["County Name"],fips_file["FIPS County"]))

In [6]:
print(ut_fips_dict)

{'Beaver': '001', 'Box Elder': '003', 'Cache': '005', 'Carbon': '007', 'Daggett': '009', 'Davis': '011', 'Duchesne': '013', 'Emery': '015', 'Garfield': '017', 'Grand': '019', 'Iron': '021', 'Juab': '023', 'Kane': '025', 'Millard': '027', 'Morgan': '029', 'Piute': '031', 'Rich': '033', 'Salt Lake': '035', 'San Juan': '037', 'Sanpete': '039', 'Sevier': '041', 'Summit': '043', 'Tooele': '045', 'Uintah': '047', 'Utah': '049', 'Wasatch': '051', 'Washington': '053', 'Wayne': '055', 'Weber': '057'}


In [7]:
print(ut_election.head())

  county         precinct       office  district    candidate party votes
0  Wayne          Fremont  U.S. Senate       NaN  Tim Aalders  Con      7
1  Wayne       Loa in/out  U.S. Senate       NaN  Tim Aalders  Con      6
2  Wayne     Lyman in/out  U.S. Senate       NaN  Tim Aalders  Con      6
3  Wayne  Bicknell in/out  U.S. Senate       NaN  Tim Aalders  Con      8
4  Wayne         Teesdale  U.S. Senate       NaN  Tim Aalders  Con      3


In [8]:
print(ut_election["county"].unique())
ut_election["county"]=ut_election["county"].map(ut_fips_dict).fillna(ut_election["county"])
print(ut_election["county"].unique())

['Wayne' 'Salt Lake' 'Wasatch' 'Iron' 'Beaver' 'Utah' 'Carbon' 'Juab'
 'Kane' 'Garfield' 'Daggett' 'Sanpete' 'Sevier' 'Box Elder' 'Grand'
 'Millard' 'San Juan' 'Tooele' 'Duchesne' 'Cache' 'Weber' 'Davis' 'Summit'
 'Uintah' 'Emery' 'Piute' 'Washington' 'Rich' 'Morgan']
['055' '035' '051' '021' '001' '049' '007' '023' '025' '017' '009' '039'
 '041' '003' '019' '027' '037' '045' '013' '005' '057' '011' '043' '047'
 '015' '031' '053' '033' '029']


In [9]:
print(ut_election["office"].unique())
office_changes_dict = {'U S Senate':'U.S. Senate'}
ut_election["office"] = ut_election["office"].map(office_changes_dict).fillna(ut_election["office"])
print(ut_election["office"].unique())

['U.S. Senate' 'U.S. House' 'State House' 'State Senate'
 'Registered Voters' 'Ballots Cast' 'Straight Party' 'U S Senate'
 'U S House' 'State Senate ' 'Utah State House']
['U.S. Senate' 'U.S. House' 'State House' 'State Senate'
 'Registered Voters' 'Ballots Cast' 'Straight Party' 'U S House'
 'State Senate ' 'Utah State House']


In [10]:
ut_election = ut_election[ut_election["office"]=='U.S. Senate']
print(len(ut_election["county"].unique()))

29


In [11]:
print(ut_election["candidate"].unique())

['Tim Aalders' 'Craig Bowden' 'Reed McCandless' 'Jenny Wilson'
 'Mitt Romney' 'write-in' 'MITT ROMNEY' 'JENNY WILSON' 'CRAIG R.  BOWDEN'
 'TIM AALDERS' 'REED C. MCCANDLESS' 'WRITE-IN' 'CALEB DAN REEVE'
 'TYRONE JENSEN' 'GLADE G FITZGERALD' 'HEKTOR REIKSTHEGN' 'CODY JUDY'
 'RYAN DANIEL JACKSON' 'ABE KORB' 'Ryan Jackson' 'Craig R. Bowden'
 'Reed C. McCandless' 'Glade G Fitzgerald (W)' 'Caleb Dan Reeve (W)'
 'Hektor Reiksthegn (W)' 'Abe Korb (W)' 'Ryan Daniel Jackson (W)'
 'Cody Judy (W)' 'Tyrone Jensen (W)' 'Write In' 'Write-In Totals'
 'write-ins' 'Write-In Total' 'CRAIG R. BOWDEN' 'REED C. McCANDLESS'
 'Write-ins' 'Write-In' 'CRAIG R BOWDEN' 'REED C MCCANDLESS'
 'Caleb Dan Reeve' 'Write-In Votes']


In [12]:
ut_election["candidate"] = ut_election["candidate"].str.title()

G18USSRROM - Mitt Romney (Republican Party)
G18USSDWIL - Jenny Wilson (Democratic Party)
G18USSIMCC - Reed C. McCandless (Independent American Party)
G18USSLBOW - Craig R. Bowden (Libertarian Party)
G18USSCAAL - Tim Aalders (Conservative Party)
G18USSOWRI - Write-in Votes

In [13]:
print(ut_election["candidate"].unique())

['Tim Aalders' 'Craig Bowden' 'Reed Mccandless' 'Jenny Wilson'
 'Mitt Romney' 'Write-In' 'Craig R.  Bowden' 'Reed C. Mccandless'
 'Caleb Dan Reeve' 'Tyrone Jensen' 'Glade G Fitzgerald'
 'Hektor Reiksthegn' 'Cody Judy' 'Ryan Daniel Jackson' 'Abe Korb'
 'Ryan Jackson' 'Craig R. Bowden' 'Glade G Fitzgerald (W)'
 'Caleb Dan Reeve (W)' 'Hektor Reiksthegn (W)' 'Abe Korb (W)'
 'Ryan Daniel Jackson (W)' 'Cody Judy (W)' 'Tyrone Jensen (W)' 'Write In'
 'Write-In Totals' 'Write-Ins' 'Write-In Total' 'Craig R Bowden'
 'Reed C Mccandless' 'Write-In Votes']


In [14]:
cand_name_change_dict = {'Craig R.  Bowden':'Craig Bowden',
 'Craig R. Bowden':'Craig Bowden',
 'Craig R Bowden':'Craig Bowden',
 'Reed C. Mccandless':'Reed Mccandless',
'Ryan Daniel Jackson':'Ryan Jackson',
'Reed C Mccandless':'Reed Mccandless',
'Write-In Votes':'Write-In',
'Write-In Total':'Write-In',
'Write-Ins':'Write-In',
'Write-In Totals':'Write-In',
'Write In':'Write-In',
'Glade G Fitzgerald (W)':'Glade G Fitzgerald',
 'Caleb Dan Reeve (W)':'Caleb Dan Reeve',
'Hektor Reiksthegn (W)':'Hektor Reiksthegn', 
'Abe Korb (W)':'Abe Korb',
 'Ryan Daniel Jackson (W)':'Ryan Jackson',
'Cody Judy (W)':'Cody Judy',
'Tyrone Jensen (W)':'Tyrone Jensen'}

In [15]:
ut_election["candidate"] = ut_election["candidate"].map(cand_name_change_dict).fillna(ut_election["candidate"])

In [16]:
print(ut_election["candidate"].unique())

['Tim Aalders' 'Craig Bowden' 'Reed Mccandless' 'Jenny Wilson'
 'Mitt Romney' 'Write-In' 'Caleb Dan Reeve' 'Tyrone Jensen'
 'Glade G Fitzgerald' 'Hektor Reiksthegn' 'Cody Judy' 'Ryan Jackson'
 'Abe Korb']


In [17]:
print(ut_election)

      county         precinct       office  district        candidate party  \
0        055          Fremont  U.S. Senate       NaN      Tim Aalders  Con    
1        055       Loa in/out  U.S. Senate       NaN      Tim Aalders  Con    
2        055     Lyman in/out  U.S. Senate       NaN      Tim Aalders  Con    
3        055  Bicknell in/out  U.S. Senate       NaN      Tim Aalders  Con    
4        055         Teesdale  U.S. Senate       NaN      Tim Aalders  Con    
...      ...              ...          ...       ...              ...   ...   
41987    029     Croydon 15.5  U.S. Senate       NaN     Craig Bowden   LIB   
41988    029     Croydon 15.5  U.S. Senate       NaN  Reed Mccandless   IAP   
41989    029     Croydon 15.5  U.S. Senate       NaN     Jenny Wilson   DEM   
41990    029     Croydon 15.5  U.S. Senate       NaN      Mitt Romney   REP   
41991    029     Croydon 15.5  U.S. Senate       NaN         Write-In   NaN   

      votes  
0         7  
1         6  
2        

In [18]:
ut_election["pivot_col"]=ut_election["county"]+ut_election["precinct"]

In [19]:
pivoted_2018 = pd.pivot_table(ut_election,values=["votes"],index=["pivot_col"],columns=["candidate"],aggfunc=sum)


In [20]:
print(pivoted_2018)

                 votes                                         \
candidate     Abe Korb Caleb Dan Reeve Cody Judy Craig Bowden   
pivot_col                                                       
001Beaver 1        NaN             NaN       NaN            9   
001Beaver 2        NaN             NaN       NaN           10   
001Beaver 3        NaN             NaN       NaN            2   
001Beaver 4        NaN             NaN       NaN            4   
001Greenville      NaN             NaN       NaN            1   
...                ...             ...       ...          ...   
057WHV007          NaN             NaN       NaN           11   
057WHV008          NaN             NaN       NaN            9   
057WNO001          NaN             NaN       NaN            *   
057WNO002          NaN             NaN       NaN            *   
057WNO003          NaN             NaN       NaN            0   

                                                                             \
candidate 

In [21]:
pivoted_2018=pivoted_2018.fillna(0)
print(pivoted_2018.head())

                 votes                                         \
candidate     Abe Korb Caleb Dan Reeve Cody Judy Craig Bowden   
pivot_col                                                       
001Beaver 1          0               0         0            9   
001Beaver 2          0               0         0           10   
001Beaver 3          0               0         0            2   
001Beaver 4          0               0         0            4   
001Greenville        0               0         0            1   

                                                                             \
candidate     Glade G Fitzgerald Hektor Reiksthegn Jenny Wilson Mitt Romney   
pivot_col                                                                     
001Beaver 1                    0                 0           46         402   
001Beaver 2                    0                 0           46         310   
001Beaver 3                    0                 0           30         244   
001Be

In [22]:
pivoted_2018.reset_index(drop=False,inplace=True)
pivoted_2018.columns = pivoted_2018.columns.droplevel(0)
print(pivoted_2018.head())

candidate                Abe Korb Caleb Dan Reeve Cody Judy Craig Bowden  \
0            001Beaver 1        0               0         0            9   
1            001Beaver 2        0               0         0           10   
2            001Beaver 3        0               0         0            2   
3            001Beaver 4        0               0         0            4   
4          001Greenville        0               0         0            1   

candidate Glade G Fitzgerald Hektor Reiksthegn Jenny Wilson Mitt Romney  \
0                          0                 0           46         402   
1                          0                 0           46         310   
2                          0                 0           30         244   
3                          0                 0            7          84   
4                          0                 0            9          56   

candidate Reed Mccandless Ryan Jackson Tim Aalders Tyrone Jensen Write-In  
0               

In [23]:
print(pivoted_2018.columns)

Index(['', 'Abe Korb', 'Caleb Dan Reeve', 'Cody Judy', 'Craig Bowden',
       'Glade G Fitzgerald', 'Hektor Reiksthegn', 'Jenny Wilson',
       'Mitt Romney', 'Reed Mccandless', 'Ryan Jackson', 'Tim Aalders',
       'Tyrone Jensen', 'Write-In'],
      dtype='object', name='candidate')


In [ ]:
G18USSRROM - Mitt Romney (Republican Party)
G18USSDWIL - Jenny Wilson (Democratic Party)
G18USSIMCC - Reed C. McCandless (Independent American Party)
G18USSLBOW - Craig R. Bowden (Libertarian Party)
G18USSCAAL - Tim Aalders (Conservative Party)
G18USSOWRI - Write-in Votes

In [24]:
pivoted_2018.columns = ["pivot_col","write_1","write_2","write_3","G18USSLBOW",
                       "write_4","write_5","G18USSDWIL",
                       "G18USSRROM","G18USSIMCC","write_6","G18USSCAAL",
                       "write_7","write_8"]

In [25]:
print(pivoted_2018)

          pivot_col write_1 write_2 write_3 G18USSLBOW write_4 write_5  \
0       001Beaver 1       0       0       0          9       0       0   
1       001Beaver 2       0       0       0         10       0       0   
2       001Beaver 3       0       0       0          2       0       0   
3       001Beaver 4       0       0       0          4       0       0   
4     001Greenville       0       0       0          1       0       0   
...             ...     ...     ...     ...        ...     ...     ...   
2266      057WHV007       0       0       0         11       0       0   
2267      057WHV008       0       0       0          9       0       0   
2268      057WNO001       0       0       0          *       0       0   
2269      057WNO002       0       0       0          *       0       0   
2270      057WNO003       0       0       0          0       0       0   

     G18USSDWIL G18USSRROM G18USSIMCC write_6 G18USSCAAL write_7 write_8  
0            46        402          

In [30]:
#What are these asteriks doing!
fips_file["County Name"].reset_index(drop=True,inplace=True)

In [35]:
county_num_dict = dict(zip(fips_file["FIPS County"],fips_file["County Name"].index+1))
                      


In [41]:
pivoted_2018["county_fips"]=pivoted_2018["pivot_col"].str[0:3]
pivoted_2018["county_num"] = 0
pivoted_2018["county_num"] = pivoted_2018["county_fips"].map(county_num_dict).fillna(pivoted_2018["county_num"])

In [51]:
pivoted_2018["county_num"] = pivoted_2018["county_num"].astype(str)

In [52]:
pivoted_2018["join_col"]=pivoted_2018["county_num"]+pivoted_2018["pivot_col"].str[3:]

In [57]:
vest_ut_18["CountyID"]=vest_ut_18["CountyID"].astype(str)

In [58]:
vest_ut_18["join_col"]=vest_ut_18["CountyID"]+vest_ut_18["mergeid"]

In [59]:
join_attempt_one = pd.merge(vest_ut_18,pivoted_2018,how="outer",on="join_col",indicator=True)
print(join_attempt_one["_merge"].value_counts())

right_only = join_attempt_one[join_attempt_one["_merge"]=="right_only"]
left_only = join_attempt_one[join_attempt_one["_merge"]=="left_only"]

left_only.to_csv("./only_vest.csv")
right_only.to_csv("./only_source.csv")


both          1693
left_only      644
right_only     578
Name: _merge, dtype: int64


In [60]:
source_to_vest_name_changes = pd.read_csv("./source_to_vest_name_changes.csv")

In [61]:
print(source_to_vest_name_changes)

         source_name vest_name
0          1Beaver 1     1BV01
1          1Beaver 2     1BV02
2          1Beaver 3     1BV03
3          1Beaver 4     1BV04
4        1Greenville     1GR01
5         1Milford 1     1ML01
6         1Milford 2     1ML02
7         1Milford 3     1ML03
8       1Minersville     1MV01
9   2Brigham City 01     2BC01
10  2Brigham City 02     2BC02
11  2Brigham City 03     2BC03
12  2Brigham City 04     2BC04
13  2Brigham City 05     2BC05
14  2Brigham City 06     2BC06
15  2Brigham City 07     2BC07
16  2Brigham City 08     2BC08
17  2Brigham City 09     2BC09
18  2Brigham City 10     2BC10
19  2Brigham City 11     2BC11
20  2Brigham City 12     2BC12
21  2Brigham City 13     2BC13
22  2Brigham City 14     2BC14
23  2Brigham City 15     2BC15


In [65]:
source_to_vest_name_changes_dict = dict(zip(source_to_vest_name_changes["source_name"],source_to_vest_name_changes["vest_name"]))
pivoted_2018["join_col"] = pivoted_2018["join_col"].map(source_to_vest_name_changes_dict).fillna(pivoted_2018["join_col"])

In [67]:
join_attempt_one = pd.merge(vest_ut_18,pivoted_2018,how="outer",on="join_col",indicator=True)
print(join_attempt_one["_merge"].value_counts())

right_only = join_attempt_one[join_attempt_one["_merge"]=="right_only"]
left_only = join_attempt_one[join_attempt_one["_merge"]=="left_only"]
both = join_attempt_one[join_attempt_one["_merge"]=="both"]

left_only.to_csv("./only_vest.csv")
right_only.to_csv("./only_source.csv")
both.to_csv("./both.csv")

both          1717
left_only      620
right_only     554
Name: _merge, dtype: int64
